In [1]:
import tensorflow as tf
import numpy as np

In [2]:
import cv2

class Preprocessor:
    def __init__(self):
        pass
    
    def begin(self, image):
        if self.image is not None:
            raise Error("image is already set. Explicitly call done() for each preprocessor")
        self.image = image
        return self
    
    def resize(self, w, h):
        self.image = cv2.resize(self.image, (w, h)) 
        return self
    
    def grayscale(self):
        self.image = cv2.cvtColor(self.image, cv2.COLOR_RGB2GRAY) 
        return self
    
    def done():
        img = self.image
        self.image = None
        return self.image

In [36]:
class NN:
    
    class Initializers:
        @staticmethod
        def ortho_init(scale=1.0):
            def _ortho_init(shape, dtype, partition_info=None):
                #lasagne ortho init for tf
                shape = tuple(shape)
                if len(shape) == 2:
                    flat_shape = shape
                elif len(shape) == 4: # assumes NHWC
                    flat_shape = (np.prod(shape[:-1]), shape[-1])
                else:
                    raise NotImplementedError
                a = np.random.normal(0.0, 1.0, flat_shape)
                u, _, v = np.linalg.svd(a, full_matrices=False)
                q = u if u.shape == flat_shape else v # pick the one with the correct shape
                q = q.reshape(shape)
                return (scale * q[:shape[0], :shape[1]]).astype(np.float32)
            return _ortho_init
    
    class Layers:
        @staticmethod
        def convolution(x, scope, filters, kernel, stride, pad='VALID', init_scale=1.0, one_dim_bias=False):
            channel_ax = 3
            strides = [1] + stride + [1]
            bshape = [1, 1, 1, filters]

            bias_var_shape = [filters] if one_dim_bias else [1, filters, 1, 1]
            nin = x.get_shape()[channel_ax].value
            wshape = kernel + [nin, filters]
            
            with tf.variable_scope(scope):
                w = tf.get_variable("w", wshape, initializer=NN.Initializers.ortho_init(init_scale))
                b = tf.get_variable("b", bias_var_shape, initializer=tf.constant_initializer(0.0))
                if not one_dim_bias:
                    b = tf.reshape(b, bshape)
                return b + tf.nn.conv2d(x, w, strides=strides, padding=pad, data_format='NHWC')

        @staticmethod
        def fully_connected(x, scope, nh, init_scale=1.0, init_bias=0.0):
            """
            x = input
            scope = name
            nh = number of hidden neurons
            """
            with tf.variable_scope(scope):
                nin = x.get_shape()[1].value
                w = tf.get_variable("w", [nin, nh], initializer=NN.Initializers.ortho_init(init_scale))
                b = tf.get_variable("b", [nh], initializer=tf.constant_initializer(init_bias))
                return tf.matmul(x, w)+b
            
        @staticmethod
        def flatten(x):
            nh = np.prod([v.value for v in x.get_shape()[1:]])
            x = tf.reshape(x, [-1, nh])
            return x


In [37]:
class Actor:

    def __init__(self, sess, ob_space, ac_space, nbatch):
        self.model = self._build(sess, ob_space, ac_space, nbatch)

    def _build(self, sess, ob_space, ac_space, nbatch):
        print(ob_space.shape)
        ob_shape = (nbatch, ) + ob_space.shape
        
        X = tf.placeholder(tf.float32, ob_shape) # Observation
        print(X.get_shape())
        
        with tf.variable_scope('model', reuse=tf.AUTO_REUSE):
            # Convolutional Layer #1
            hidden_1 = tf.nn.relu(NN.Layers.convolution(X, "conv_1", 32, [8, 8], [4, 4], init_scale=np.sqrt(2)))
            
            # Convolutional Layer #2
            hidden_2 = tf.nn.relu(NN.Layers.convolution(hidden_1, "conv_2", 64, [4, 4], [2, 2], init_scale=np.sqrt(2)))
            
            # Convolutional Layer #3
            hidden_3 = tf.nn.relu(NN.Layers.convolution(hidden_2, "conv_3", 64, [3, 3], [1, 1], init_scale=np.sqrt(2)))
            
            # Flatten to FC
            flatten = NN.Layers.flatten(hidden_3)
            
            # Classification Layer # 1
            fc_1 = tf.nn.relu(NN.Layers.fully_connected(flatten, 'fc_1', 512, init_scale=np.sqrt(2)))

            
sess = tf.get_default_session()
Actor(sess, np.zeros(shape=(80, 80, 3)), np.zeros(shape=(4, )), 32)

            
            
            
    
            

(80, 80, 3)
(32, 80, 80, 3)


In [35]:
class Critic:

    def __init__(self):
        self.model = self._build()

    def _build(self):
        pass

In [ ]:
class Agent:
    pass